In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    AutoConfig,
)
from pytorch_lightning import LightningModule
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained('./out-small-1.1')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
tokenizer.push_to_hub('nanot5-small-malaysian-cased', organization='mesolitica')

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/utils/hub.py:700: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/mesolitica/nanot5-small-malaysian-cased/commit/955d4e0bb7f44c4b6f6566158cd86b6d9f8b80d2', commit_message='Upload tokenizer', commit_description='', oid='955d4e0bb7f44c4b6f6566158cd86b6d9f8b80d2', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
class Module(LightningModule):
    def __init__(self):
        super().__init__()
        config = AutoConfig.from_pretrained(
            './out-small-1.1'
        )
        self.model = T5ForConditionalGeneration.from_pretrained(
            './out-small-1.1',
            config=config,
        )

In [6]:
!ls logs/small

'model-epoch=04-step=42000.ckpt'  'model-epoch=04-step=46000.ckpt'
'model-epoch=04-step=44000.ckpt'


In [7]:
model = Module()

In [8]:
weights = model.state_dict()

In [9]:
old_weights = torch.load('logs/small/model-epoch=04-step=46000.ckpt',
                             map_location=torch.device('cpu'))['state_dict'].items()

In [10]:
for k, v in old_weights:
    new_k = k.replace('._orig_mod', '')
    print(k, new_k)
    weights[new_k] = v

model.shared.weight model.shared.weight
model.encoder.embed_tokens.weight model.encoder.embed_tokens.weight
model.encoder.block.0.layer.0.SelfAttention.q.weight model.encoder.block.0.layer.0.SelfAttention.q.weight
model.encoder.block.0.layer.0.SelfAttention.k.weight model.encoder.block.0.layer.0.SelfAttention.k.weight
model.encoder.block.0.layer.0.SelfAttention.v.weight model.encoder.block.0.layer.0.SelfAttention.v.weight
model.encoder.block.0.layer.0.SelfAttention.o.weight model.encoder.block.0.layer.0.SelfAttention.o.weight
model.encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight model.encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight
model.encoder.block.0.layer.0.layer_norm.weight model.encoder.block.0.layer.0.layer_norm.weight
model.encoder.block.0.layer.1.DenseReluDense.wi_0.weight model.encoder.block.0.layer.1.DenseReluDense.wi_0.weight
model.encoder.block.0.layer.1.DenseReluDense.wi_1.weight model.encoder.block.0.layer.1.DenseReluDense.wi_1.w

In [11]:
model.load_state_dict(weights)

<All keys matched successfully>

In [12]:
model.model.push_to_hub('nanot5-small-malaysian-cased', organization='mesolitica')

pytorch_model.bin:   0%|          | 0.00/320M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/nanot5-small-malaysian-cased/commit/676d1c93831723cd4766fc4370381d4922a44701', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='676d1c93831723cd4766fc4370381d4922a44701', pr_url=None, pr_revision=None, pr_num=None)